In [3]:
import calculate_mean_ap

In [4]:
from copy import deepcopy
import json
import glob
import os
import time
import argparse
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# import seaborn as sns
#
# sns.set_style('white')
# sns.set_context('poster')
from pygments.lexer import default



COLORS = [
    '#1f77b4', '#aec7e8', '#ff7f0e', '#ffbb78', '#2ca02c',
    '#98df8a', '#d62728', '#ff9896', '#9467bd', '#c5b0d5',
    '#8c564b', '#c49c94', '#e377c2', '#f7b6d2', '#7f7f7f',
    '#c7c7c7', '#bcbd22', '#dbdb8d', '#17becf', '#9edae5']

In [5]:
ground_truth_boxes = '/home/vgoncalves/personal-git/people-detection-compare/resources/virat_dataset/VIRAT_videos_moving_ground_truth_boxes.json'

In [6]:
predicted_boxes = '/home/vgoncalves/personal-git/people-detection-compare/results/VIRAT_videos_haar_moving_predicted_boxes.json'

In [7]:
with open(ground_truth_boxes) as infile:
    gt_boxes = json.load(infile)

with open(predicted_boxes) as infile:
    pred_boxes = json.load(infile)

In [8]:
#pred_boxes

In [9]:
model_scores_map = calculate_mean_ap.get_model_scores_map(pred_boxes)
sorted_model_scores = sorted(model_scores_map.keys())

In [10]:
sorted_model_scores

[1]

In [11]:
for img_id in pred_boxes.keys():
    arg_sort = np.argsort(pred_boxes[img_id]['scores'])
    pred_boxes[img_id]['scores'] = np.array(pred_boxes[img_id]['scores'])[arg_sort].tolist()
    pred_boxes[img_id]['boxes'] = np.array(pred_boxes[img_id]['boxes'])[arg_sort].tolist()

In [12]:
pred_boxes

{'VIRAT_S_000201_02_000590_000623_frame_1': {'boxes': [[243, 48, 275, 112],
   [218, 482, 246, 538]],
  'scores': [1, 1]},
 'VIRAT_S_000201_02_000590_000623_frame_2': {'boxes': [[244, 47, 275, 109]],
  'scores': [1]},
 'VIRAT_S_000201_02_000590_000623_frame_3': {'boxes': [[244, 49, 276, 112],
   [218, 482, 247, 540]],
  'scores': [1, 1]},
 'VIRAT_S_000201_02_000590_000623_frame_4': {'boxes': [[244, 52, 275, 114],
   [470, 61, 506, 134],
   [217, 482, 246, 540]],
  'scores': [1, 1, 1]},
 'VIRAT_S_000201_02_000590_000623_frame_5': {'boxes': [[244, 49, 276, 112],
   [217, 52, 248, 114],
   [217, 481, 247, 541]],
  'scores': [1, 1, 1]},
 'VIRAT_S_000201_02_000590_000623_frame_6': {'boxes': [[244, 50, 275, 112],
   [217, 49, 249, 112],
   [468, 58, 507, 137],
   [217, 482, 246, 540]],
  'scores': [1, 1, 1, 1]},
 'VIRAT_S_000201_02_000590_000623_frame_7': {'boxes': [[243, 50, 275, 113],
   [217, 53, 248, 114],
   [270, 56, 301, 118],
   [218, 482, 246, 538]],
  'scores': [1, 1, 1, 1]},
 'VIR

In [13]:
pred_boxes_pruned = deepcopy(pred_boxes)

In [14]:
pred_boxes_pruned

{'VIRAT_S_000201_02_000590_000623_frame_1': {'boxes': [[243, 48, 275, 112],
   [218, 482, 246, 538]],
  'scores': [1, 1]},
 'VIRAT_S_000201_02_000590_000623_frame_2': {'boxes': [[244, 47, 275, 109]],
  'scores': [1]},
 'VIRAT_S_000201_02_000590_000623_frame_3': {'boxes': [[244, 49, 276, 112],
   [218, 482, 247, 540]],
  'scores': [1, 1]},
 'VIRAT_S_000201_02_000590_000623_frame_4': {'boxes': [[244, 52, 275, 114],
   [470, 61, 506, 134],
   [217, 482, 246, 540]],
  'scores': [1, 1, 1]},
 'VIRAT_S_000201_02_000590_000623_frame_5': {'boxes': [[244, 49, 276, 112],
   [217, 52, 248, 114],
   [217, 481, 247, 541]],
  'scores': [1, 1, 1]},
 'VIRAT_S_000201_02_000590_000623_frame_6': {'boxes': [[244, 50, 275, 112],
   [217, 49, 249, 112],
   [468, 58, 507, 137],
   [217, 482, 246, 540]],
  'scores': [1, 1, 1, 1]},
 'VIRAT_S_000201_02_000590_000623_frame_7': {'boxes': [[243, 50, 275, 113],
   [217, 53, 248, 114],
   [270, 56, 301, 118],
   [218, 482, 246, 538]],
  'scores': [1, 1, 1, 1]},
 'VIR

In [15]:
sorted_model_scores[:-1]

[]

In [16]:
precisions = []
recalls = []
model_thrs = []
img_results = {}

# Loop over model score thresholds and calculate precision, recall
for ithr, model_score_thr in enumerate(sorted_model_scores):
    # On first iteration, define img_results for the first time:
    img_ids = gt_boxes.keys() if ithr == 0 else model_scores_map[model_score_thr]
    for img_id in img_ids:
        gt_boxes_img = gt_boxes[img_id]
        #print(img_id)
        
        box_scores = pred_boxes_pruned[img_id]['scores']
        #print(box_scores)
        start_idx = 0
        for score in box_scores:
            if score <= model_score_thr:
                pred_boxes_pruned[img_id]
                start_idx += 1
            else:
                break

#         # Remove boxes, scores of lower than threshold scores:
#         pred_boxes_pruned[img_id]['scores'] = pred_boxes_pruned[img_id]['scores'][start_idx:]
#         pred_boxes_pruned[img_id]['boxes'] = pred_boxes_pruned[img_id]['boxes'][start_idx:]

        # Recalculate image results for this image
        img_results[img_id] = calculate_mean_ap.get_single_image_results(
            gt_boxes_img, pred_boxes_pruned[img_id]['boxes'], 0.5)

    prec, rec = calculate_mean_ap.calc_precision_recall(img_results)
    precisions.append(prec)
    recalls.append(rec)
    model_thrs.append(model_score_thr)


In [54]:
pred_boxes_pruned['VIRAT_S_000201_02_000590_000623_frame_3']['scores']

[1, 1]

In [20]:
img_results

{'VIRAT_S_000201_02_000590_000623_frame_1': {'false_neg': 4,
  'false_pos': 2,
  'true_pos': 0},
 'VIRAT_S_000201_02_000590_000623_frame_2': {'false_neg': 4,
  'false_pos': 1,
  'true_pos': 0},
 'VIRAT_S_000201_02_000590_000623_frame_3': {'false_neg': 4,
  'false_pos': 2,
  'true_pos': 0},
 'VIRAT_S_000201_02_000590_000623_frame_4': {'false_neg': 4,
  'false_pos': 3,
  'true_pos': 0},
 'VIRAT_S_000201_02_000590_000623_frame_5': {'false_neg': 4,
  'false_pos': 3,
  'true_pos': 0},
 'VIRAT_S_000201_02_000590_000623_frame_6': {'false_neg': 4,
  'false_pos': 4,
  'true_pos': 0},
 'VIRAT_S_000201_02_000590_000623_frame_7': {'false_neg': 4,
  'false_pos': 4,
  'true_pos': 0},
 'VIRAT_S_000201_02_000590_000623_frame_8': {'false_neg': 4,
  'false_pos': 4,
  'true_pos': 0},
 'VIRAT_S_000201_02_000590_000623_frame_9': {'false_neg': 4,
  'false_pos': 3,
  'true_pos': 0},
 'VIRAT_S_000201_02_000590_000623_frame_10': {'false_neg': 4,
  'false_pos': 3,
  'true_pos': 0},
 'VIRAT_S_000201_02_000590_00

In [17]:
pred_boxes_pruned

{'VIRAT_S_000201_02_000590_000623_frame_1': {'boxes': [[243, 48, 275, 112],
   [218, 482, 246, 538]],
  'scores': [1, 1]},
 'VIRAT_S_000201_02_000590_000623_frame_2': {'boxes': [[244, 47, 275, 109]],
  'scores': [1]},
 'VIRAT_S_000201_02_000590_000623_frame_3': {'boxes': [[244, 49, 276, 112],
   [218, 482, 247, 540]],
  'scores': [1, 1]},
 'VIRAT_S_000201_02_000590_000623_frame_4': {'boxes': [[244, 52, 275, 114],
   [470, 61, 506, 134],
   [217, 482, 246, 540]],
  'scores': [1, 1, 1]},
 'VIRAT_S_000201_02_000590_000623_frame_5': {'boxes': [[244, 49, 276, 112],
   [217, 52, 248, 114],
   [217, 481, 247, 541]],
  'scores': [1, 1, 1]},
 'VIRAT_S_000201_02_000590_000623_frame_6': {'boxes': [[244, 50, 275, 112],
   [217, 49, 249, 112],
   [468, 58, 507, 137],
   [217, 482, 246, 540]],
  'scores': [1, 1, 1, 1]},
 'VIRAT_S_000201_02_000590_000623_frame_7': {'boxes': [[243, 50, 275, 113],
   [217, 53, 248, 114],
   [270, 56, 301, 118],
   [218, 482, 246, 538]],
  'scores': [1, 1, 1, 1]},
 'VIR

In [18]:
precisions

[0.4323241842384262]

In [19]:
recalls

[0.07822991494894806]

In [21]:
model_thrs

[1]

In [58]:
enumerate(sorted_model_scores)

In [59]:
img_ids = gt_boxes.keys() if ithr == 0 else model_scores_map[1]

In [23]:
img_ids

dict_keys(['VIRAT_S_000201_02_000590_000623_frame_1', 'VIRAT_S_000201_02_000590_000623_frame_2', 'VIRAT_S_000201_02_000590_000623_frame_3', 'VIRAT_S_000201_02_000590_000623_frame_4', 'VIRAT_S_000201_02_000590_000623_frame_5', 'VIRAT_S_000201_02_000590_000623_frame_6', 'VIRAT_S_000201_02_000590_000623_frame_7', 'VIRAT_S_000201_02_000590_000623_frame_8', 'VIRAT_S_000201_02_000590_000623_frame_9', 'VIRAT_S_000201_02_000590_000623_frame_10', 'VIRAT_S_000201_02_000590_000623_frame_11', 'VIRAT_S_000201_02_000590_000623_frame_12', 'VIRAT_S_000201_02_000590_000623_frame_13', 'VIRAT_S_000201_02_000590_000623_frame_14', 'VIRAT_S_000201_02_000590_000623_frame_15', 'VIRAT_S_000201_02_000590_000623_frame_16', 'VIRAT_S_000201_02_000590_000623_frame_17', 'VIRAT_S_000201_02_000590_000623_frame_18', 'VIRAT_S_000201_02_000590_000623_frame_19', 'VIRAT_S_000201_02_000590_000623_frame_20', 'VIRAT_S_000201_02_000590_000623_frame_21', 'VIRAT_S_000201_02_000590_000623_frame_22', 'VIRAT_S_000201_02_000590_0006

In [28]:
for img_id in img_ids:
    gt_boxes_img = gt_boxes[img_id]
    box_scores = pred_boxes_pruned[img_id]['scores']
    start_idx = 0
    for score in box_scores:
        if score <= model_score_thr:
            pred_boxes_pruned[img_id]
            start_idx += 1
        else:
            break
            
# Remove boxes, scores of lower than threshold scores:
pred_boxes_pruned[img_id]['scores'] = pred_boxes_pruned[img_id]['scores'][start_idx:]
pred_boxes_pruned[img_id]['boxes'] = pred_boxes_pruned[img_id]['boxes'][start_idx:]

# Recalculate image results for this image
img_results[img_id] = calculate_mean_ap.get_single_image_results(
    gt_boxes_img, pred_boxes_pruned[img_id]['boxes'], 0.5)

In [25]:
box_scores

[1, 1]

In [29]:
img_results

{'VIRAT_S_050000_12_001591_001619_frame_833': {'false_neg': 3,
  'false_pos': 0,
  'true_pos': 0}}

In [37]:


# precisions = []
# recalls = []
# model_thrs = []
# img_results = {}
# # Loop over model score thresholds and calculate precision, recall
# for ithr, model_score_thr in enumerate(sorted_model_scores):
#     # On first iteration, define img_results for the first time:
#     img_ids = gt_boxes.keys() if ithr == 0 else model_scores_map[model_score_thr]
#     for img_id in img_ids:
#         gt_boxes_img = gt_boxes[img_id]
#         box_scores = pred_boxes_pruned[img_id]['scores']
#         start_idx = 0
#         for score in box_scores:
#             if score <= model_score_thr:
#                 pred_boxes_pruned[img_id]
#                 start_idx += 1
#             else:
#                 break

#         # Remove boxes, scores of lower than threshold scores:
#         pred_boxes_pruned[img_id]['scores'] = pred_boxes_pruned[img_id]['scores'][start_idx:]
#         pred_boxes_pruned[img_id]['boxes'] = pred_boxes_pruned[img_id]['boxes'][start_idx:]

#         # Recalculate image results for this image
#         img_results[img_id] = calculate_mean_ap.get_single_image_results(
#             gt_boxes_img, pred_boxes_pruned[img_id]['boxes'], 0.5)

#     prec, rec = calculate_mean_ap.calc_precision_recall(img_results)
#     precisions.append(prec)
#     recalls.append(rec)
#     model_thrs.append(model_score_thr)


In [31]:
sorted_model_scores[0]

1

In [36]:
precisions

[0.0]

In [37]:
recalls

[0.0]